This is a jupyter notebook version of an implementation of a Linear-Quadratic Regulator (LQR) in python for solving the OpenAI Gym environment of CartPole. This notebook will only give a brief-overview of LQR, and if you are interested in digging deeper into control theory, I highly suggest watching Steve Brunton's "Control Bootcamp" videos (link below). This notebook may be confusing otherwise, but watching the lecture series will give you all the background information you need, and it's really good! If you want to see the full implementation, just jump to the bottom of this notebook, or look at the associated python file in this folder!

![LQR](./gym_animation.gif)



[Above] Example implementation where random actions are taken at the start, and then LQR controller takes over and stabalizes the cartpole

https://www.youtube.com/watch?v=Pi7l8mMjYVE&ab_channel=SteveBrunton

Anyways, lets jump into it. First, let's take a look at the cartpole domain in OpenAI gym. What we can first do is just spin up the environment, take random actions, and see how long it stays up. In this case, I'll set a time out of max number of steps that it can take, and we will stop the simulation when the pole falls over (done is True) or we hit the timeout number of steps.

In [3]:
import gym
import numpy as np

env = gym.make('CartPole-v1')
timeout = 500

observation = env.reset()
total_time = 0
while True:
    #uncomment below if you want to render environment
    #env.render()
    action = env.action_space.sample() #sample random action
    observation, reward, done, info = env.step(action)
    total_time += 1
    if done or total_time == timeout:
        break
print("Total number of steps:",total_time)
    

Total number of steps: 27


Here, when the timeout is 500 and we take random actions, we don't get even close to staying up that long, generally it seems to take a total number of steps less than 100 before falling. Now, the question is, can we do better than this? Rather than use a learning based approach, we will instead use LQR, which requires no learning at all, but does require some assumptions about the dynamics of the problem and our initial setting of the cartpole state. Let's jump into it.

First, let's look at the state space of the problem. Every observation can be broken into 4 state variables: the position of the cart, the velocity of the cart, the angle of the pole, and the angular velocity of the pole.

In [8]:
# observation: state. 4D in this case.
x, v, theta, v_theta = observation

The action space for this cartpole domain is actually discrete (sending a 0 applies a leftward force on the cart, sending a 1 applies a rightward force). However, LQR will give us a feedback controller that gives continuous actions. We'll deal with that later, but for now, the important thing to know is that we will treat the action space as being 1D.

Next, we need to define the dynamics of the system using a system of linear equations (this is what the L stands for in LQR). Unfortunately, the actual dynamics of the cartpole problem are nonlinear. Therefore, what we can do is linearize the cartpole dynamics at the upright unstable equilibrium point, and use that as our approximation to the system dynamics. When the system is fully-controlable, this gurantees that we can derive a linear feedback controller that will drive us to the unstable equilibrium point. However, if the state deviates too far away from the equilibrium point, our controller may not be able to recover. 

Writing out the state dynamics of a physical system is generally not-trivial, and I will not get into the details here on how do that and linearize around a certain point. For our purposes, I used the following link, which included a description of the linear dynamics of the cartpole at the unstable upright position: https://metr4202.uqcloud.net/tpl/t8-Week13-pendulum.pdf

The equation for the dynamics requires knowing four constants: the mass of the cart, the mass of the pole, the length of the pole, and the force of gravity. All of these variables are accessible from inside the gym environment.

In [9]:
M = float(env.masscart)
m = float(env.masspole)
l = float(env.length)
g = float(env.gravity)


Now, based on the equations from the previous link, we can write out the A and B matrix for LQR by plugging in these variables. For this problem, we know that A should be a 4x4 matrix, and B should be a 4x1 matrix.


In [10]:
# linearization
#A = np.identity(4)
A = [[0,1,0,0],[0,0,-1*(m*g)/M,0],[0,0,0,1],[0,0,((M+m)*g)/(l*M),0]]
A = np.array(A)

#B = np.ones((4,1))
B = [[0],[1/M],[0],[-1/(l*M)]]
B = np.array(B)

Now we need to define the cost matrices for the state and action, which will define our quadratic cost function (the Q in LQR). For simplicity, we will give equal penalty weighting to all state variables and action dimensions, so we just use the identity matrix. Since the state space is 4d and the action space is 1D, Q and R will be square matrices of the same size respectively.

In [11]:
Q = np.identity(4)
R = np.identity(1)

Now, we've got everything we need (a description of the dynamics that is linear in the state and action, and a cost function that is quadratic in the state and action). We can now just use the LQR solver from the control library of python to get the optimal state feedback controller. More information can be found at the following link: https://python-control.readthedocs.io/en/0.8.1/generated/control.lqr.html

In [12]:
import control

K, S, E = control.lqr(A,B,Q,R)

K is the state feedback gains, S is the solution to the Riccati equation, and E is the eigenvalues of the closed systems. As long as the real parts of the eigenvalues are all negative, then we have a stable controller. Let's take a look:

In [13]:
print(E)

[-5.7559314+0.j         -3.7642922+0.j         -0.8106413+0.49745536j
 -0.8106413-0.49745536j]


Looks good to me! All we actually really need is K, because it defines the feedback controller which is linear in the state.

In [14]:
action = -1*np.dot(K,observation)

This gives us a 1d continuous action to take, but the cartpole we're working with is discrete action (0 relates to moving leftware, 1 refers to moving rightward). So, we'll just take a simple thresholding based on the sign of the action to decide which discrete action to take:

In [15]:
if action >= 0:
    print("action is positive",action)
    action = 1
else:
    print("action is negative",action)
    action = 0

action is negative [[-13.20366438]]


Let's package this all into one function, that will take in the observation, and use LQR to decide the action:

In [6]:
def lqr_policy(observation):
    M = float(env.masscart)
    m = float(env.masspole)
    l = float(env.length)
    g = float(env.gravity)
    # observation: state. 4D in this case.
    x, v, theta, v_theta = observation
    # cost function 

    Q = np.identity(4)
    R = np.identity(1)

    # linearization
    #A = np.identity(4)
    A = [[0,1,0,0],[0,0,-1*(m*g)/M,0],[0,0,0,1],[0,0,((M+m)*g)/(l*M),0]]
    A = np.array(A)

    #B = np.ones((4,1))
    B = [[0],[1/M],[0],[-1/(l*M)]]
    B = np.array(B)

    #K (2-d array) – State feedback gains: ???
    #S (2-d array) – Solution to Riccati equation
    #E (1-d array) – Eigenvalues of the closed loop system
    #print("A:",A.shape)
    #print("B:",B.shape)
    #print("Q:",Q.shape)
    #print("R:",R.shape)
    K, S, E = control.lqr(A,B,Q,R)
    #print("K:",K)
    #print("S:",S)
    #print("E:",E)
    #print("Observation:",observation)
    action = -1*np.dot(K,observation)
    if action >= 0:
        return(1)
    else:
        return(0)


Since we're not changing A,B,Q,R between time steps, we don't need to continually recompute K each time step, but instead compute K once at the start and use it to define our linear feedback controller, but it's pretty fast in this domain so it's not a big issue to do the computation over and over again.

Now let's look at what happens when we substitue in LQR for the random policy:

In [23]:
import gym
import numpy as np

env = gym.make('CartPole-v1')
timeout = 500

observation = env.reset()
total_time = 0
while True:
    #uncomment below if you want to render environment
    env.render()
    action = lqr_policy(observation) #sample random action
    observation, reward, done, info = env.step(action)
    total_time += 1
    if done or total_time == timeout:
        break
print("Total number of steps:",total_time)

Total number of steps: 500


Now we're consistently hitting the max number of time steps allowed by the timeout, and if you render the environment, you can watch the cartpole stabalize! It's important to remember that we linearized around the unstable, upright equilibrium point, which means that as our state diverges from that equilibrium, our approximation to the dynamics gets worse and worse, making our controller worse and worse. When the cartpole starts about upright, like when we do reset, this is no problem, but once the cartpole falls over, it can't recover. As long as our approximation to the dynamics is reasonable, our controller should be able to recover. For example, let's say that for the first 10 steps, we act according to the random policy, and then start doing LQR?

In [15]:
import gym
import numpy as np
import control


env = gym.make('CartPole-v1')
timeout = 500

observation = env.reset()
total_time = 0
while True:
    #uncomment below if you want to render environment
    #env.render()
    action = env.action_space.sample() if total_time <= 10 else lqr_policy(observation) #sample random action
    observation, reward, done, info = env.step(action)
    total_time += 1
    if done or total_time == timeout:
        break
print("Total number of steps:",total_time)

Total number of steps: 500


In general, even when we take 10 random actions, it seems to recover and get back to the stable point. But if we take too many random actions, like 50, recovery becomes highly unlikely

In [19]:
import gym
import numpy as np
import control


env = gym.make('CartPole-v1')
timeout = 500

observation = env.reset()
total_time = 0
while True:
    #uncomment below if you want to render environment
    #env.render()
    action = env.action_space.sample() if total_time <= 50 else lqr_policy(observation) #sample random action
    observation, reward, done, info = env.step(action)
    total_time += 1
    if done or total_time == timeout:
        break
print("Total number of steps:",total_time)

Total number of steps: 11


You can play around with Q and R to set different costs for the state and action, which will change how aggresive the controller is.